# Intro
This is a notebook what is used for the articles on my blog. The main intention to write the series of articles how to use python in game designer work. This will be updated.

# Simple Fight Game

The players summon different units, they move to each other and fight.

## 1:1 Fight
For the fight we want to use simulation to determine the outcome of the fight. Instead of using just formulas we are building simplified game-loop. Lets say units start on opposite sides of the field. Move to each other and attacks when its possible.
For that we need a way to save their state. For that we just use a dictionary for simplicity. This notebook covers 1:1 fight.

## Implementation

The current implementation and chosed data structures are not the best. But they are simple and easy to understand for first steps. The main goal is to show how to use python in game designer work. The code is not optimized and only for learning purposes.

In [5]:
# libraries and imports
from collections import namedtuple

# Some init constants and unit attributes
# For unit names we are using 'hyphen-case'
# For unit attributes we are using 'snake_case'
UNIT_TYPES = ['soldier', 'knight', 'archer', 'goblin', 'ork', 'axe-thrower']
# health - maximum health points
# attack_damage - damage per attack
# speed - how fast unit can move, m/s
# attack_range - how far unit can attack, m
# attack_cooldown - how often unit can attack, s
ATTRIBUTE_NAMES = ['health', 'attack_damage', 'speed', 'attack_range', 'attack_cooldown']

# Named tuple for unit attributes
# https://docs.python.org/3/library/collections.html#collections.namedtuple
Unit = namedtuple('Unit', ATTRIBUTE_NAMES)

# Units attributes
UNITS_ATTRIBUTES = {
    'soldier': Unit(health=100, attack_damage=10, speed=1.5, attack_range=1, attack_cooldown=0.7),
    'knight': Unit(health=150, attack_damage=15, speed=1.2, attack_range=1, attack_cooldown=1),
    'archer': Unit(health=50, attack_damage=20, speed=1, attack_range=10, attack_cooldown=2),
    'goblin': Unit(health=50, attack_damage=10, speed=1.5, attack_range=1, attack_cooldown=0.5),
    'ork': Unit(health=100, attack_damage=15, speed=1.2, attack_range=1, attack_cooldown=1),
    'axe-thrower': Unit(health=50, attack_damage=20, speed=1, attack_range=10, attack_cooldown=2),
}

In [8]:
def create_unit(unit_type: str, x=0) -> dict:
    attributes = UNITS_ATTRIBUTES[unit_type]
    unit = {}
    unit['type'] = unit_type
    unit['max_health'] = attributes.health
    unit['current_health'] = attributes.health
    unit['current_attack_cooldown'] = 0
    unit['x'] = x
    return unit

def get_unit_attribute(unit: dict, attribute: str):
    return getattr(UNITS_ATTRIBUTES[unit['type']], attribute)


In [10]:
DT = 0.1
FIELD_SIZE = 15

def distance_between(unit1: dict, unit2: dict):
    return abs(unit1['x'] - unit2['x'])

def enemy_in_range(unit: dict, enemy: dict):
    return distance_between(unit, enemy) <= get_unit_attribute(unit, "attack_range")

def unit_can_attack(unit: dict):
    return unit["current_attack_cooldown"] <= 0

def simulate_fight(unit_type1: dict, unit_type2: dict):
    unit1 = create_unit(unit_type1, 0)
    unit2 = create_unit(unit_type2, FIELD_SIZE)
    time = 0
    while unit1["current_health"] > 0 and unit2["current_health"] > 0:
        if enemy_in_range(unit1, unit2):
            if unit_can_attack(unit1):
                unit2["current_health"] -= get_unit_attribute(unit1, "attack_damage")
                unit1["current_attack_cooldown"] = get_unit_attribute(unit1, "attack_cooldown")
        else:
            unit1["x"] += get_unit_attribute(unit1, "speed") * DT
        if enemy_in_range(unit2, unit1):
            if unit_can_attack(unit2):
                unit1["current_health"] -= get_unit_attribute(unit2, "attack_damage")
                unit2["current_attack_cooldown"] = get_unit_attribute(unit2, "attack_cooldown")
        else:
            unit2["x"] -= get_unit_attribute(unit2, "speed") * DT
        time += DT
        unit1["current_attack_cooldown"] = max(0, unit1["current_attack_cooldown"] - DT)
        unit2["current_attack_cooldown"] = max(0, unit2["current_attack_cooldown"] - DT)
    return time, unit1, unit2




In [35]:
def print_results(results):
    for time, unit1, unit2 in results:
        print('-' * 50)
        time = round(time, 1)
        print(f'Fight between {unit1["type"]} and {unit2["type"]} lasted {time} seconds')
        if unit1['current_health'] > 0:
            print(f'Winner: {unit1["type"]} with remaining health {unit1["current_health"]}')
        elif unit2['current_health'] > 0:
            print(f'Winner: {unit2["type"]} with remaining health {unit2["current_health"]}')
        else:
            print('Draw, both units died')

# Results are presented as a list of tuples (time, unit1, unit2)
# we want to print it as ASCII table where rows and columns are unit types
# cells are remaining health of unit1 after fight
def print_results_table(results):
    # first we need to find all unique unit types
    INTENT = 12
    unit_types = set()
    for _, unit1, unit2 in results:
        unit_types.add(unit1['type'])
        unit_types.add(unit2['type'])
    unit_types = sorted(list(unit_types))
    # print header
    print(' ' * 14, end='')
    for unit_type in unit_types:
        print(f'{unit_type:>14}', end='')
    print()
    # print rows
    for unit_type1 in unit_types:
        print(f'{unit_type1:>14}', end='')
        for unit_type2 in unit_types:
            # find result for this pair of unit types
            for time, u1, u2 in results:
                if u1['type'] == unit_type1 and u2['type'] == unit_type2:
                    break
            else:
                raise Exception(f'No result for {unit_type1} vs {unit_type2}')
            # print cell
            hp = max(u1['current_health'], 0)
            if hp == 0:
                print(f'\x1b[31m{hp:14.0f}\x1b[0m', end='')
            else:
                print(f'\x1b[32m{hp:14.0f}\x1b[0m', end='')
        print()

results = []
for unit_type1 in UNIT_TYPES:
    for unit_type2 in UNIT_TYPES:
        t, u1, u2 = simulate_fight(unit_type1, unit_type2)
        # print_results(t, u1, u2)
        results.append((t, u1, u2))

print_results_table(results)
print_results(results)


                      archer   axe-thrower        goblin        knight           ork       soldier
        archer             0             0            50             0            35            20
   axe-thrower             0             0            50             0            35            20
        goblin             0             0             0             0             0             0
        knight            30            30            90             0            45            60
           ork             0             0            40             0             0            10
       soldier             0             0            40             0             0             0
--------------------------------------------------
Fight between soldier and soldier lasted 12.0 seconds
Draw, both units died
--------------------------------------------------
Fight between soldier and knight lasted 11.9 seconds
Winner: knight with remaining health 60
------------------------------------